# NOTE
**Please ensure that you have ran the *Mitchel and YouTuBean train test split* notebooks first so that all of the datasets are avaliable**

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import os
from collections import defaultdict
from pathlib import Path

import pandas as pd
import numpy as np

# Helper functions
from bella.notebook_helper import write_json_data, get_json_data
# Models
from bella.models.target import TargetDep
from bella.models.target import TargetDepSent
# Word Vector methods
from bella.word_vectors import GloveCommonCrawl, SSWE
from bella.helper import read_config
# Sentiment lexicons
from bella import lexicons
# Get the data
from bella.parsers import semeval_14, dong, election
from bella.data_types import TargetCollection
# Evaluation methods
from bella.evaluation import evaluation_results, get_results, get_raw_data
# Tokenisers
from bella.tokenisers import ark_twokenize

Using TensorFlow backend.


In [2]:
def fine_tune_values(c_values):
    fine_values = []
    for c_value in c_values:
        if '35' in c_value or '7' in c_value:
            fine_values.append(float(c_value))
    fine_values = sorted(fine_values)
    best_coarse_c_value = fine_values[3] / 7
    fine_values.append(best_coarse_c_value)
    return fine_values, best_coarse_c_value
def coarse_tune_values(c_values):
    coarse_values = []
    fine_values = fine_tune_values(c_values)
    for c_value in c_values:
        c_value = float(c_value)
        if c_value not in fine_values:
            coarse_values.append(c_value)
    return coarse_values
def best_c_value(c_values):
    best = 0
    best_c = 0
    for c_value, acc in c_values.items():
        if acc > best:
            best_c = c_value
            best = acc
    return best_c

In [3]:
##
#  ADD YOUR CONFIG FILE PATH HERE 
##
CONFIG_FP = Path('..', 'config.yaml')

# Load all of the datasets
youtubean_train = semeval_14(read_config('youtubean_train', CONFIG_FP))
youtubean_test = semeval_14(read_config('youtubean_test', CONFIG_FP))
semeval_14_rest_train = semeval_14(read_config('semeval_2014_rest_train', CONFIG_FP))
semeval_14_lap_train = semeval_14(read_config('semeval_2014_lap_train', CONFIG_FP))
semeval_14_rest_test = semeval_14(read_config('semeval_2014_rest_test', CONFIG_FP))
semeval_14_lap_test = semeval_14(read_config('semeval_2014_lap_test', CONFIG_FP))
dong_train = dong(read_config('dong_twit_train_data', CONFIG_FP))
dong_test = dong(read_config('dong_twit_test_data', CONFIG_FP))
election_train, election_test = election(read_config('election_folder_dir', CONFIG_FP))
mitchel_train = semeval_14(read_config('mitchel_train', CONFIG_FP))
mitchel_test = semeval_14(read_config('mitchel_test', CONFIG_FP))


dataset_train_test = {'SemEval 14 Laptop' : (semeval_14_lap_train, semeval_14_lap_test),
                      'SemEval 14 Restaurant' : (semeval_14_rest_train, semeval_14_rest_test),
                      'Dong Twitter' : (dong_train, dong_test),
                      'Election Twitter' : (election_train, election_test),
                      'YouTuBean' : (youtubean_train, youtubean_test),
                      'Mitchel' : (mitchel_train, mitchel_test)
                     }

In [4]:
# Load the sentiment lexicons and remove all words that are not associated
# to the Positive or Negative class.

lexicon_data = read_config('lexicons', CONFIG_FP)
hu_liu_fp = Path(lexicon_data['hu_liu'])
mpqa_fp = Path(lexicon_data['mpqa'])
nrc_fp = Path(lexicon_data['nrc_emotion'])

subset_cats = {'positive', 'negative'}

# Load the sentiment lexicons but lower case all the words
mpqa_low = lexicons.Mpqa(mpqa_fp, subset_cats=subset_cats, lower=True)
nrc_low = lexicons.NRC(nrc_fp, subset_cats=subset_cats, lower=True)
hu_liu_low = lexicons.HuLiu(hu_liu_fp, subset_cats=subset_cats, lower=True)
mpqa_huliu_low = lexicons.Lexicon.combine_lexicons(mpqa_low, hu_liu_low)
all_three_low = lexicons.Lexicon.combine_lexicons(mpqa_huliu_low, nrc_low)

In [5]:
def dataset_predictions(train, test, dataset_name, model,
                        random_state,c_file_path, word_vector_file_path, model_dir,
                        sentiment_lexicon=None, result_file_path=None,
                        re_write=True, save_raw_data=True):
    # Gets the results dataframe if it already exists
    if not re_write and result_file_path is not None:
        results_df = get_results(result_file_path, name)
        if save_raw_data and results_df is not None:
            if get_raw_data(result_file_path, name, test):
                return results_df
        elif results_df is not None:
            return results_df
    # loading the data
    data_train = train.data()
    y_train = train.sentiment_data()
    data_test = test.data()
    y_test = test.sentiment_data()
    # Reduce the word vector size by filtering by the word in the train
    # and test dataset. Which is still as fair in practice as loading the whole
    # word vectors

    train_words = train.word_list(ark_twokenize)
    test_words = test.word_list(ark_twokenize)
    all_words = list(set(train_words + test_words))
    glove_300 = GloveCommonCrawl(version=42, filter_words=all_words)
    sswe = SSWE(filter_words=all_words)
    
    # CV grid params
    c_grid_params = {'word_vectors' : [[sswe]], 'random_state' : random_state,
                     'tokenisers' : [ark_twokenize]}
    if sentiment_lexicon is not None:
        c_grid_params['senti_lexicons'] = [sentiment_lexicon]
        
    best_c, c_scores = model.find_best_c(data_train, y_train, c_grid_params, 
                                         save_file=c_file_path, dataset_name=dataset_name, 
                                         re_write=False, n_jobs=7, cv=5)
    
    # Search over the different word vectors 
    word_vectors = [[sswe], [glove_300]]
    word_vector_grid_params = {**c_grid_params}
    word_vector_grid_params['C'] = [best_c]
    word_vector_grid_params['word_vectors'] = word_vectors
    import time
    t = time.time()
    best_word_vector = model.save_grid_search(data_train, y_train, word_vector_grid_params, 
                                              'word_vectors', dataset_name, word_vector_file_path, 
                                              re_write=False, n_jobs=5, cv=5)
    print('{} {}'.format(best_word_vector, time.time() - t))
    
    parameters = {'word_vector' : best_word_vector, 'random_state' : random_state, 
                  'C' : best_c, 'tokeniser' : ark_twokenize}
    if sentiment_lexicon is not None:
        parameters['senti_lexicon'] = sentiment_lexicon
    best_params = model.get_params(**parameters)
    print('Best parameters for dataset {} are: {}'.format(dataset_name, parameters))
    model.fit(data_train, y_train, params=best_params)
    predicted_values = model.predict(data_test)
    # Save the model to the model zoo
    model_file_name = '{} {}'.format(model, dataset_name)
    model_file_path = os.path.join(model_dir, model_file_name)
    model.save_model(model_file_path, verbose=1)
    # Return the results
    if result_file_path is not None:
        return evaluation_results(predicted_values, test, dataset_name, 
                                  file_name=result_file_path, 
                                  save_raw_data=save_raw_data, re_write=True)
    else:
        return evaluation_results(predicted_values, test, dataset_name)
   

In [6]:
# Instances of the models
target_dep = TargetDep()
target_dep_plus = TargetDepSent()
models = [target_dep, target_dep_plus]

# Target dependent model mass evaluation

The above code loads all of the data, models, and lexicons we are going to use in this notebook

We are going to use three different models.
1. target_dep -- Target Dependent model that uses no sentiment lexicons
2. target_dep_plus -- Target Dependent model that uses only the Hu & Liu lexicon
3. target_dep_plus_all -- Target Dependent model that uses all three lexicons from the original paper

Each model gets it's own results file where it will store the results from each dataset.

In [ ]:
# Creating the result files
result_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'results_alt', 'Target Dependent'))
model_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'model zoo alt'))
os.makedirs(result_folder, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
model_result_files = ['Target Dependent.tsv', 'Target Dependent+.tsv']
model_result_files = [os.path.join(result_folder, result_file) for result_file in model_result_files]
C_result_files = ['Target Dependent C.json', 'Target Dependent+ C.json']
C_result_files = [os.path.join(result_folder, result_file) for result_file in C_result_files]
word_vector_result_files = ['Target Dependent word vector.json', 'Target Dependent+ word vector.json']
word_vector_result_files = [os.path.join(result_folder, result_file) for result_file in word_vector_result_files]
# Parameters for each model
std_model_parameters = {'random_state' : 42}
all_senti_model_parameters = {**std_model_parameters, 'sentiment_lexicon' : all_three_low}
model_parameters = [std_model_parameters, all_senti_model_parameters]
# Combining parameters and result files
parameters_files = list(zip(model_parameters, model_result_files, C_result_files, 
                            word_vector_result_files, [model_dir]*2))

model_files = dict(zip(models, parameters_files))
model_files

{Target Dependent: ({'random_state': 42},
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/results_alt/Target Dependent/Target Dependent.tsv',
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/results_alt/Target Dependent/Target Dependent C.json',
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/results_alt/Target Dependent/Target Dependent word vector.json',
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/model zoo alt'),
 Target Dependent Plus: ({'random_state': 42,
   'sentiment_lexicon': <bella.lexicons.Lexicon at 0x7f84ad0a3358>},
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/results_alt/Target Dependent/Target Dependent+.tsv',
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/results_alt/Target Dependent/Target Dependent+ C.json',
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/results_alt/Target Dependent/Target Dependent+ word vector.json',
  '/mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/model zoo alt')}

In [ ]:
import time
time_to_process = time.time()
for dataset_name, train_test in dataset_train_test.items():
    print('Processing dataset {}'.format(dataset_name))
    train, test = train_test
    for model, parameter_file_paths in model_files.items():
        print('Processing model {}'.format(model))
        params_files = parameter_file_paths
        parameters = params_files[0]
        result_file_path = params_files[1]
        c_fp = params_files[2]
        word_vectors_fp = params_files[3]
        model_dir = params_files[4]
        dataset_predictions(train, test, dataset_name, model, 
                            result_file_path=result_file_path,
                            re_write=True, save_raw_data=True,
                            c_file_path=c_fp,
                            word_vector_file_path=word_vectors_fp, 
                            model_dir=model_dir,
                            **parameters)

time_to_process = time.time() - time_to_process

Processing dataset SemEval 14 Laptop
Processing model Target Dependent
Loading glove 300d 42b common crawl from file
[glove 300d 42b common crawl] 99.16763806343079
Best parameters for dataset SemEval 14 Laptop are: {'word_vector': [glove 300d 42b common crawl], 'random_state': 42, 'C': 0.01, 'tokeniser': <function ark_twokenize at 0x7f84b9358840>}
Model saved to /mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/model zoo alt/Target Dependent SemEval 14 Laptop. Save time 0.64
saving raw data
Processing model Target Dependent Plus
Loading glove 300d 42b common crawl from file
[glove 300d 42b common crawl] 117.39861941337585
Best parameters for dataset SemEval 14 Laptop are: {'word_vector': [glove 300d 42b common crawl], 'random_state': 42, 'C': 0.0035, 'tokeniser': <function ark_twokenize at 0x7f84b9358840>, 'senti_lexicon': <bella.lexicons.Lexicon object at 0x7f84ad0a3358>}
Model saved to /mnt/silo/andrew/Sentiment_Models/EMNLP/Bella/model zoo alt/Target Dependent Plus SemEval 14 Laptop. S

Time it took to process all the datasets {{round(time_to_process / 3600, 2)}} hours

## The affect of the C value

We take a look at the affect of tunning for the C-Value. We tune the C-Value in two steps:
1. Coarse grain search over the following values: from 0.00001 to 10 going up by a factor of 10. Once the best best coarse grain value is found we fine tune it.
2. We multiple the best coarse grain value by the following value and search over these values: 0.35, 0.7, 1, 3.5, and 7.

In [ ]:
c_values_target_dep_plus = C_result_files[1]
c_values_target_dep_plus = get_json_data(c_values_target_dep_plus, 'Dong Twitter')

In [ ]:
dataset_stats = defaultdict(lambda: dict())

mean_error = lambda acc_values: sum(max(acc_values) - acc_values) / (len(acc_values) - 1)
for dataset_name in dataset_train_test:
    # Target Dependent Plus using the Hu and Liu Lexicon C file
    c_values_target_dep_plus = C_result_files[1]
    c_values_target_dep_plus = get_json_data(c_values_target_dep_plus, dataset_name)
    model_results_file = model_result_files[1]
    fine_c_values, best_coarse_c_value = fine_tune_values(c_values_target_dep_plus)
    coarse_c_values = coarse_tune_values(c_values_target_dep_plus)
    all_c_values = {float(c_value) : acc for c_value, acc in c_values_target_dep_plus.items()}
    fine_acc = np.array([all_c_values[c_value] for c_value in fine_c_values])
    coarse_acc = np.array([all_c_values[c_value] for c_value in coarse_c_values])
    all_acc = np.array(list(c_values_target_dep_plus.values()))
    best_c = best_c_value(all_c_values)
    test_accuracy = get_results(model_results_file, dataset_name)['Accuracy']
    dataset_stats[dataset_name]['Fine Tune MAE (Accuracy %)'] = mean_error(fine_acc) * 100
    dataset_stats[dataset_name]['Coarse Tune MAE (Accuracy %)'] = mean_error(coarse_acc) * 100
    dataset_stats[dataset_name]['Best Fine C value'] = best_c
    dataset_stats[dataset_name]['Best Coarse C value'] = best_coarse_c_value
    dataset_stats[dataset_name]['Test score Accuracy (%)'] = test_accuracy

In [ ]:
index = list(dataset_train_test)
columns = ['Fine Tune MAE (Accuracy %)', 'Coarse Tune MAE (Accuracy %)', 'Best Fine C value', 
           'Best Coarse C value', 'Dataset', 'Test score Accuracy (%)']
c_value_stats = pd.DataFrame(np.zeros((len(dataset_train_test), 6)), columns=columns)
c_value_stats['Dataset'] = index
c_value_stats = c_value_stats.set_index('Dataset')

# Add the data to the DataFrame
for dataset_name, col_value in dataset_stats.items():
    for column, value in col_value.items():
        c_value_stats[column][dataset_name] = value
c_value_stats = c_value_stats.round({'Fine Tune Accuracy std (%)' : 2, 
                                     'Coarse Tune Accuracy std (%)' : 2})
c_value_stats